In [28]:
using Units

┌ Info: Recompiling stale cache file /Users/jlizarraga/.julia/compiled/v1.0/Units/fBxJC.ji for Units [e118339a-c993-11e8-3139-b11bd8eb6de4]
└ @ Base loading.jl:1187


In [33]:
struct SIBaseUnit{m,kg,s,A,K,mol,cd} #<: SIBaseUnits
    SIBaseUnit{m,kg,s,A,K,mol,cd}() where {m,kg,s,A,K,mol,cd} = isa((m,kg,s,A,K,mol,cd), NTuple{7,Int}) ? new() : error("An SIBaseUnit{m,kg,s,A,K,mol,cd} can only be constructed with integers")
end
# struct SIBaseUnit{m<:Int,kg<:Int,s<:Int,A<:Int,K<:Int,mol<:Int,cd<:Int} expects m <: Type{Int} where typeof(m)::DataType NOT isa(m,Int)

ErrorException: syntax: invalid variable expression in "where"

In [1]:
struct SIBaseUnit{T <: Array{Int,7}} #<: SIBaseUnits
    
end

In [5]:
siArray = ["m","kg","s","A","K","mol","cd"]
siTuple = tuple(Symbol.(siArray.*"_i")...)
siUnit  = :(SIBaseUnit{$(siTuple...)})

siArray1 = siArray.*"1"
siTuple1 = tuple(Symbol.(siArray1)...)
siUnit1  = :(SIBaseUnit{$(siTuple1...)})

siArray2 = siArray.*"2"
siTuple2 = tuple(Symbol.(siArray2)...)
siUnit2  = :(SIBaseUnit{$(siTuple2...)})

siTuple1Tuple2 = (siTuple1...,siTuple2...)
siTupleResult = Dict(:* => :([$(Symbol.(siArray1)...)] .+ [$(Symbol.(siArray2)...)]), # -> :([m1, kg1, s1, A1, K1, mol1, cd1] .+ [m2, kg2, s2, A2, K2, mol2, cd2])
                     :/ => :([$(Symbol.(siArray1)...)] .- [$(Symbol.(siArray2)...)]),
                     :\ => :([$(Symbol.(siArray2)...)] .- [$(Symbol.(siArray1)...)]))

Dict{Symbol,Expr} with 3 entries:
  :\ => :([m2, kg2, s2, A2, K2, mol2, cd2] .- [m1, kg1, s1, A1, K1, mol1, cd1])
  :/ => :([m1, kg1, s1, A1, K1, mol1, cd1] .- [m2, kg2, s2, A2, K2, mol2, cd2])
  :* => :([m1, kg1, s1, A1, K1, mol1, cd1] .+ [m2, kg2, s2, A2, K2, mol2, cd2])

In [13]:
siTupleResult = Dict(:* => Meta.parse(join(siArray1 .* "+" .* siArray2, ",")), # -> :((m1 + m2, kg1 + kg2, s1 + s2, A1 + A2, K1 + K2, mol1 + mol2, cd1 + cd2))
                     :/ => Meta.parse(join(siArray1 .* "-" .* siArray2, ",")),
                     :\ => Meta.parse(join(siArray2 .* "-" .* siArray1, ",")))

Dict{Symbol,Expr} with 3 entries:
  :\ => :((m2 - m1, kg2 - kg1, s2 - s1, A2 - A1, K2 - K1, mol2 - mol1, cd2 - cd…
  :/ => :((m1 - m2, kg1 - kg2, s1 - s2, A1 - A2, K1 - K2, mol1 - mol2, cd1 - cd…
  :* => :((m1 + m2, kg1 + kg2, s1 + s2, A1 + A2, K1 + K2, mol1 + mol2, cd1 + cd…

In [15]:
print(typeof(siArray1 .* "+" .* siArray2))
print(siArray1 .* "+" .* siArray2)

Array{String,1}["m1+m2", "kg1+kg2", "s1+s2", "A1+A2", "K1+K2", "mol1+mol2", "cd1+cd2"]

In [8]:
# For either case the following code works:

for op in (:*, :/, :\)
   @eval $op( ::Type{$siUnit1}, ::Type{$siUnit2}) where {$(siTuple1Tuple2...)} = SIBaseUnit{$(siTupleResult[op])...}
   @eval $op( ::Type{$siUnit1},y::$siUnit2      ) where {$(siTuple1Tuple2...)} = SIBaseUnit{$(siTupleResult[op])...}
   @eval $op(x::$siUnit1      , ::Type{$siUnit2}) where {$(siTuple1Tuple2...)} = SIBaseUnit{$(siTupleResult[op])...}
   @eval $op(x::$siUnit1      ,y::$siUnit2      ) where {$(siTuple1Tuple2...)} = SIBaseUnit{$(siTupleResult[op])...}
end

ErrorException: error in method definition: function Base.* must be explicitly imported to be extended

In [11]:
# This code ...
for i in 1:2
   siArrayX, siTupleX, siUnitX  = Symbol("siArray"*repr(i)), Symbol("siTuple"*repr(i)), Symbol("siUnit" *repr(i))
   @eval $siArrayX = siArray.*$(repr(i))
   @eval $siTupleX = tuple(Symbol.($siArrayX)...)
   @eval $siUnitX  = :(SIBaseUnit{$(($siTupleX)...)})
end

# Produces this code
siArray1 = siArray.*"1"
siTuple1 = tuple(Symbol.(siArray1)...)
siUnit1  = :(SIBaseUnit{$(siTuple1...)})

siArray2 = siArray.*"2"
siTuple2 = tuple(Symbol.(siArray2)...)
siUnit2  = :(SIBaseUnit{$(siTuple2...)})

In [12]:
siUnit3

:(SIBaseUnit{m3, kg3, s3, A3, K3, mol3, cd3})

In [ ]:
macro generateshowcode(suffix, tupleOfUnits...) # ... denotes a variable number of arguments.
   expressionsArray = []
   for unit in tupleOfUnits
      functionParameter = Symbol(unit,suffix)
      ex = esc(:($functionParameter ≠ 0 && print(io, $(String(unit)), ($functionParameter == 1 ? ' ' : tosuperscript($functionParameter)))))
      push!(expressionsArray, ex)
   end

   return quote $(tuple(expressionsArray...)...) end
end

   # The code above generates the code under when called with @generateshowcode("_i", m,kg,s,A,K,mol,cd)
   #   m_i ≠ 0 && print(io, "m"  , (  m_i == 1 ? ' ' : tosuperscript(  m_i)))
   #  kg_i ≠ 0 && print(io, "kg" , ( kg_i == 1 ? ' ' : tosuperscript( kg_i)))
   #   s_i ≠ 0 && print(io, "s"  , (  s_i == 1 ? ' ' : tosuperscript(  s_i)))
   #   A_i ≠ 0 && print(io, "A"  , (  A_i == 1 ? ' ' : tosuperscript(  A_i)))
   #   K_i ≠ 0 && print(io, "K"  , (  K_i == 1 ? ' ' : tosuperscript(  K_i)))
   # mol_i ≠ 0 && print(io, "mol", (mol_i == 1 ? ' ' : tosuperscript(mol_i)))
   #  cd_i ≠ 0 && print(io, "cd" , ( cd_i == 1 ? ' ' : tosuperscript( cd_i)))

In [16]:
siArrayBase    = ["m","kg","s","A","K","mol","cd"]
suffixVariable = [""  ,"1","2"]
suffixTuple    = ["_i","1","2"]

for i in 1:length(suffixVariable)
   siArrayX, siTupleX, siUnitX  = Symbol("siArray"*suffixVariable[i]), Symbol("siTuple"*suffixVariable[i]), Symbol("siUnit" *suffixVariable[i])
   @eval $siArrayX = siArrayBase.*$(suffixTuple[i])
   @eval $siTupleX = tuple(Symbol.($siArrayX)...)
   @eval $siUnitX  = :(SIBaseUnit{$(($siTupleX)...)})
end

In [27]:
# Benchmarking

include("Angles.jl")
using Main.Angles
using Statistics
S = 10
N = 10000
t = zeros(Float64,S)
tref = zeros(Float64,S)
for i in 1:S
    s  = 2^i

    m1 = rand(s,s)
    m2 = rand(s,s)
    m3 = (m1)deg
    m4 = (m2)deg
    @elapsed m1+m2
    @elapsed m3+m4

    for j in 1:N
      tref[i] += @elapsed m1+m2
      t[i]    += @elapsed m3+m4
    end

    tref[i] /= N
    t[i] /= N

end
println(t./tref)
println(mean(t./tref))

# Results for S=10, N=1000
# [0.963113, 0.996702, 0.719338, 1.21352, 1.03335, 0.971611, 0.992234, 0.772692, 0.998298, 1.00277]
# 0.966362453606228
# Results for S=10, N=10000
# [0.987338, 0.989298, 0.497516, 1.18991, 0.931941, 0.981512, 1.00617, 1.29759, 1.00667, 0.999558]
# 0.9887505207007307

:(SIBaseUnit{m2, kg2, s2, A2, K2, mol2, cd2})

In [ ]:
# METAPROGRAMMING
#
# @eval $(ex) == eval(ex)
#
#
# AST: (call f x) -> Expr(:call, :f, :x)
# import Base: x, y	-> AST: (import (: (. Base) (. x) (. y))) -> Expr(:import, (:(:), (:., :Base), (:., :x), (:., :y)))

u = :m
m_i = 5
sy = Symbol(u,"_i")
io = stdout

@eval $(println(u,"=")) # -- m=
@macroexpand @eval $(println(u,"=")) # --> :((Base.Core).eval(Main, println(u, "=")))
@eval $(quote println(u,"=") end) # --> m=
@macroexpand @eval $(quote println(u,"=") end)  # --> :((Base.Core).eval(Main, $(Expr(:copyast, :($(QuoteNode(quote
                                                #         #= none:1 =#
                                                #         println(u, "=")
                                                #     end)))))))

@eval $(println(u,"=",sy)) # --> m=m_i
@macroexpand @eval $(println(u,"=",sy)) # :((Base.Core).eval(Main, println(u, "=", sy)))
@eval $(quote println(u,"=",$sy) end) # m=5
@macroexpand @eval $(quote println(u,"=",$sy) end) # :((Base.Core).eval(Main, (Core._expr)(:block, $(QuoteNode(:(#= none:1 =#))), (Core._expr)(:call, :println, :u, "=", sy))))
@eval $(quote println(u,"=",$(Symbol(u,"_i"))) end) # m=5
@macroexpand @eval $(quote println(u,"=",$(Symbol(u,"_i"))) end)

# IMPORTANT: The error is to put the @eval in the for-block of a FUNCTION! Remeber macros are expanded at compilation time not at runtime.s
function f(io::IO, m_i::Int, kg_i::Int, s_i::Int, A_i::Int, K_i::Int, mol_i::Int, cd_i::Int)
  for u in (:m,:kg,:s,:A,:K,:mol,:cd)
      print(@macroexpand @eval println(io,u,"="))
      print("   ")
      print(@macroexpand @eval println($io,$u,"="))
      print("   ")
      println(u)
      @eval println($io,u,"=")
  end
end

# Expressions can be cascaded in an array of expressions and then converted to a tuple for execution. Maybe a begin-block or the ";" could do the trick also.
extup = (:(x = 3),:(y = 5))
ex = quote $(extup...) end
eval(ex)

exarr = [:(x = 3),:(y = 5)]
push!(exarr, :(x+y))
ex = quote $(tuple(exarr...)...) end
eval(ex)

u = :m
m = 5
:(print(u))
:(print($u))


In [ ]:
# isdefined(Base, :__precompile__) && __precompile__()

baremodule TestModule

using Base
export teststruct

eval(x) = Core.eval(Mod, x)
include(p) = Base.include(Mod, p)

# struct teststruct{T<:Int, S<:Float64}
#   x::T
#   y::S
# end

# struct teststruct{T}
#   teststruct{T}() where T = isa(T, Int) ? new() : error("teststruct can only be constructed with integers")
# end

struct teststruct{m<:Int,kg<:Int,s<:Int,A<:Int,K<:Int,mol<:Int,cd<:Int}
  # teststruct{T}() where T = isa(T, Int) ? new() : error("teststruct can only be constructed with integers")
end
# (::Type{teststruct{t}})() where {t<:Int} = teststruct{t}()
# (::Type{teststruct{t}})() where {t>:Int} = errormessage("Error")
# teststruct(t::T) where {T<:Int} = teststruct{t}
# teststruct(t::T) where {T>:Int} = errormessage("Error")

end

using Main.TestModule
teststruct{1}()
teststruct{1.5}()

baremodule TestModule

using Base
export teststruct

eval(x) = Core.eval(Mod, x)
include(p) = Base.include(Mod, p)

teststruct = Tuple{Vararg{Int,7}}

end

using Main.TestModule
teststruct(0,0,0,0,0,0,0)

# Usage in REPL:
include("Angles.jl")
using Main.Angles
for op in Main.Angles.operationsInverse
  @eval $op(x::Real) = Main.Angles.$op(x)
end

############################

using Units
SIQuantity(0, scalar)
Units.SIQuantitiesPkg.SIQuantity(0, Units.SIBaseUnitsPkg.scalar)
SIQuanity(x::T,u::SIBaseUnit) where {T<:Number} = Units.SIQuantitiesPkg.SIQuanity{T,typeof(u)}(x)
Units.SIQuantitiesPkg.ScalarQuantity(0)

using Units

import Base: +
siArray = ["m","kg","s","A","K","mol","cd"]
siTuple = tuple(Symbol.(siArray.*"_i")...)
siUnit  = :(SIBaseUnit{$(siTuple...)})
op = :+

@eval $op(x::$siUnit,y::$siUnit) where {$(siTuple...)} = $siUnit

m()+m()

@macroexpand @eval SIBaseUnit{$(siTuple...)}
:((Base.Core).eval(Main, (Core._expr)(:quote, (Core._expr)(:curly, :SIBaseUnit, (Core._expr)(:$, $(Expr(:copyast, :($(QuoteNode(:(siTuple...)))))))))))

siTuple2 = expr(:$,QuoteNode(:(siTuple...)))
# :($(QuoteNode(:(siTuple...))))
# siTuple2 = :($(QuoteNode(:(siTuple...))))
siTuple2 = Expr(:copyast, :($(QuoteNode(:(siTuple...)))))
siTuple2 = :({$(siTuple...)})
siUnit2  = :(SIBaseUnit{$siTuple2})

@macroexpand @eval $op(x::SIBaseUnit{$siTuple},y::SIBaseUnit{$siTuple}) where {$(siTuple...)} = SIBaseUnit{$siTuple}

using Units

import Base: +
siArray = ["m","kg","s","A","K","mol","cd"]
siTuple = tuple(Symbol.(siArray.*"_i")...)
siUnit  = :(SIBaseUnit{$(siTuple...)})

siArray1 = siArray.*"1"
siTuple1 = tuple(Symbol.(siArray1)...)
siUnit1  = :(SIBaseUnit{$(siTuple1...)})

siArray2 = siArray.*"2"
siTuple2 = tuple(Symbol.(siArray2)...)
siUnit2  = :(SIBaseUnit{$(siTuple2...)})

siTuple1Tuple2 = (siTuple1...,siTuple2...)

m1 = m2 = 1
kg1=s1=A1=K1=mol1=cd1 = kg2=s2=A2=K2=mol2=cd2 = 0

ex = :([$(Symbol.(siArray1)...)] .+ [$(Symbol.(siArray2)...)])
eval(ex)

siTupleResult = Dict(:* => :([$(Symbol.(siArray1)...)] .+ [$(Symbol.(siArray2)...)]),
                     :/ => :([$(Symbol.(siArray1)...)] .- [$(Symbol.(siArray2)...)]),
                     :\ => :([$(Symbol.(siArray2)...)] .- [$(Symbol.(siArray1)...)]))

ex = (Meta.parse("("*join(siArray1 .* "+" .* siArray2, ",")*")"))
ex = Meta.parse(join(siArray1 .* "+" .* siArray2, ","))

Meta.parse(siArray1 .* "+" .* siArray2)
# siTupleResult = Dict(:* => tuple(Symbol.(siArray1 .* "+" .* siArray2)...),
#                      :/ => tuple(Symbol.(siArray1 .* "-" .* siArray2)...),
#                      :\ => tuple(Symbol.(siArray2 .* "-" .* siArray1)...))

op = :*

ex = :($op( ::Type{$siUnit1}, ::Type{$siUnit2}) where {$(siTuple1Tuple2...)} = SIBaseUnit{($siTupleResult[op])...})

@macroexpand @eval $op( ::Type{siUnit1}, ::Type{siUnit2}) where {$(siTuple1...,siTuple2...)} = SIBaseUnit{$(siTupleResult[op]...)}

ex = Meta.parse("*(x::SIBaseUnit{m1,kg1,s1,A1,K1,mol1,cd1},y::SIBaseUnit{m2,kg2,s2,A2,K2,mol2,cd2}) where {m1,kg1,s1,A1,K1,mol1,cd1,m2,kg2,s2,A2,K2,mol2,cd2} = SIBaseUnit{m1+m2,kg1+kg2,s1+s2,A1+A2,K1+K2,mol1+mol2,cd1+cd2}")


limit = 1e7
@time [^(SIUnit{1,2,3,4,5,6,7,8,9}(),20) for i=1:limit]
@time [SIBaseUnits.power(SIUnit{1,2,3,4,5,6,7,8,9}(),20) for i=1:limit]

unit = SIUnit{100,4,3,2,1,-1,-2,-3,-4}()
val = 15
quant = SIQuantity{typeof(val),1,2,3,4,5,6,7,8,9}(val)

siquantity(TypeVar(:Float64), unit)
typeof(unit)
typeof(typeof(unit))

@siquantity(Float64, unit)

val = 1.0
quant = SIQuantity{typeof(val),1,2,3,4,5,6,7,8,9}(val)
siquantity(TypeVar(:Int64), quant)

val = 100
quant = SIQuantity{typeof(val),1,2,3,4,5,6,7,8,9}(val)
@which(one(quant))
@which(one(SIQuantity{typeof(val),1,2,3,4,5,6,7,8,9}))

# macro parametersandvalues(tup...)
#     ex = esc(quote
#             parameter = $(map(x -> string(x),tup))
#             value     = $tup
#          end)
#     return ex
# end

# This macro is capable of generating specialised code when called with the "actual"
# arguments (values not parameters) at execution time. However, the value of the
# parameters is not available at parse time therefore a macro generating generic
# code is needed.
# macro buildnumandden(units, val)
#     evaluationstring = ""
#     u = eval(units)
#     v = eval(val)
#
#     for index = 1:length(v)
#         if v[index] != 0
#              leadingstring = (v[index] > 0) ?  "num" : "den"
#                 unitstring = "\\text{$(u[index])\}"
#            trainlingstring = (abs(v[index]) == 1) ? " " : "^{$(abs(v[index]))}"
#                 codestring = "push!($leadingstring,\"$(string(unitstring,trainlingstring))\")"
#           evaluationstring = string(evaluationstring, codestring, "\n")
#         end
#     end
#     #println(evaluationstring)
#     return parse("quote\n$evaluationstring\nend")
# end


include("./src/Angles.jl")
using Main.Angles
for op in Main.Angles.operationsTrigoInverse
  @eval $op(x::Real) = Main.Angles.$op(x)
end
